# Imports + class

In [8]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
from z3 import *
from xgboost import XGBClassifier
from pmlb import fetch_data
set_option(rational_to_decimal=True)

import re
import time
import pandas as pd
from model import XGBoostExplainer, generate_results
from sklearn.model_selection import train_test_split
np.set_printoptions(suppress=True)

In [9]:
class XGBoostExplainer:
    def __init__(self, model, data):
        self.model = model
        self.data = data.values
        self.columns = data.columns
        self.max_categories = 2

        set_option(rational_to_decimal=True)
        self.categoric_features = self.get_categoric_features(self.data)
        self.T_model = self.model_trees_expression(self.model)
        self.T = self.T_model

    def explain(self, instance, reorder="asc"):
        self.I = self.instance_expression(instance)
        self.D, self.D_add = self.decision_function_expression(self.model, [instance])
        return self.explain_expression(self.I, And(self.T, self.D_add), self.D, self.model, reorder)

    def explain_prob(self, instance, reorder="asc", threshold_margin=0, target_threshold=None):
        self.I = self.instance_expression(instance)
        self.D, self.D_add = self.decision_function_expression(self.model, [instance])
        return self.explain_expression_prob(self.I, And(self.T, self.D_add), self.D, self.model, reorder, threshold_margin, target_threshold)

    def get_categoric_features(self, data: np.ndarray):
        categoric_features = []
        for i in range(data.shape[1]):
            feature_values = data[:, i]
            unique_values = np.unique(feature_values)
            if len(unique_values) <= self.max_categories:
                categoric_features.append(self.columns[i])

        return categoric_features

    def feature_constraints(self, constraints=[]):
        """TODO
        esperado receber limites das features pelo usuário
        formato previso: matriz/dataframe [feaature, min/max, valor]
        constraaint_expression = "constraaint_df_to_feature()"
        """
        return

    def model_trees_expression(self, model):
        df = model.get_booster().trees_to_dataframe()
        if model.get_booster().feature_names == None:
            feature_map = {f"f{i}": col for i, col in enumerate(self.columns)}
            df["Feature"] = df["Feature"].replace(feature_map)

        df["Split"] = df["Split"].round(4)
        self.booster_df = df
        class_index = 0  # if model.n_classes_ == 2:
        all_tree_formulas = []

        for tree_index in df["Tree"].unique():
            tree_df = df[df["Tree"] == tree_index]
            o = Real(f"o_{tree_index}_{class_index}")

            if len(tree_df) == 1 and tree_df.iloc[0]["Feature"] == "Leaf":
                leaf_value = tree_df.iloc[0]["Gain"]
                all_tree_formulas.append(And(o == leaf_value))
                continue
            path_formulas = []

            def get_conditions(node_id):
                conditions = []
                current_node = tree_df[tree_df["ID"] == node_id]
                if current_node.empty:
                    return conditions

                parent_node = tree_df[
                    (tree_df["Yes"] == node_id) | (tree_df["No"] == node_id)
                ]
                if not parent_node.empty:
                    parent_data = parent_node.iloc[0]
                    feature = parent_data["Feature"]
                    split_value = parent_data["Split"]
                    x = Real(feature)
                    if parent_data["Yes"] == node_id:
                        conditions.append(x < split_value)
                    else:
                        conditions.append(x >= split_value)
                    conditions = get_conditions(parent_data["ID"]) + conditions

                return conditions

            for _, node in tree_df[tree_df["Feature"] == "Leaf"].iterrows():
                leaf_value = node["Gain"]
                leaf_id = node["ID"]
                conditions = get_conditions(leaf_id)
                path_formula = And(*conditions)
                implication = Implies(path_formula, o == leaf_value)
                path_formulas.append(implication)

            all_tree_formulas.append(And(*path_formulas))
        return And(*all_tree_formulas)

    def get_init_value(self, model, x, estimator_variables):
        estimator_pred = Solver()
        estimator_pred.add(self.I)
        estimator_pred.add(self.T)
        if estimator_pred.check() == sat:
            solvermodel = estimator_pred.model()
            total_sum = sum(
                float(solvermodel.eval(var).as_fraction()) for var in estimator_variables
            )
        else:
            total_sum = 0
            print("estimator error")
        self.predicted_margin = model.predict(x, output_margin=True)[0]
        init_value = self.predicted_margin - total_sum
        self.init_value = init_value
        return init_value

    def decision_function_expression(self, model, x):
        n_classes = 1 if model.n_classes_ <= 2 else model.n_classes_
        predicted_class = model.predict(x)[0]
        self.predicted_class = predicted_class
        n_estimators = int(len(model.get_booster().get_dump()) / n_classes)
        estimator_variables = [Real(f"o_{i}_0") for i in range(n_estimators)] # _0 only for binary classification
        self.estimator_variables = estimator_variables
        init_value = self.get_init_value(model, x, estimator_variables)
        # print("init:", round(init_value, 2))

        equation_list = []

        estimator_sum = Real("estimator_sum")
        equation_o = estimator_sum == Sum(estimator_variables)
        equation_list.append(equation_o)

        decision = Real("decision")
        equation_list.append(decision == estimator_sum + init_value)

        if predicted_class == 0:
            final_equation = decision < 0
        else:
            final_equation = decision > 0

        return final_equation, And(equation_list)

    def instance_expression(self, instance):
        formula = [Real(self.columns[i]) == value for i, value in enumerate(instance)]
        return formula

    def explain_expression(self, I, T_s, D_s, model, reorder):
        i_expression = I.copy()

        importances = model.feature_importances_
        non_zero_indices = np.where(importances != 0)[0]

        if reorder == "asc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]
        elif reorder == "desc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(-importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]

        for feature in i_expression.copy():
            # print("\n---removed", feature)
            i_expression.remove(feature)

            # prove(Implies(And(And(i_expression), T), D))
            if self.is_proved(Implies(And(And(i_expression), T_s), D_s)):
                continue
                # print('proved')
            else:
                # print('not proved')
                i_expression.append(feature)
        # print(self.is_proved(Implies(And(And(i_expression), T_s), D_s)))
        return i_expression

    def explain_expression_prob(self, I, T_s, D_s, model, reorder, threshold_margin, target_threshold):
        i_expression = I.copy()

        importances = model.feature_importances_
        non_zero_indices = np.where(importances != 0)[0]

        if reorder == "asc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]
        elif reorder == "desc":
            sorted_feature_indices = non_zero_indices[
                np.argsort(-importances[non_zero_indices])
            ]
            i_expression = [i_expression[i] for i in sorted_feature_indices]

        threshold = 0

        if target_threshold:
            threshold = target_threshold
        elif threshold_margin != 0:
            threshold = self.predicted_margin * threshold_margin/100
            # print("margin:", self.predicted_margin, "accepted margin:", threshold)
        self.xai_predicted_margin = self.predicted_margin

        for feature in i_expression.copy():
            # print("\n---removed", feature)
            i_expression.remove(feature)

            if self.is_proved_sat(And(And(i_expression), T_s), threshold):
                # print('proved')
                continue
            else:
                # print('not proved -- added back')
                i_expression.append(feature)
        return i_expression


    def is_proved(self, decision_exp):
        s = Solver()
        s.add(Not(decision_exp))
        if s.check() == unsat:
            return True
        else:
            # print(s.model())
            return False

    def is_proved_sat(self, decision_exp, threshold):
      decision = Real("decision")

      debug = Real("debug") == 0
      predicted_class = self.predicted_class

      if predicted_class == 0:
        estmax = Optimize()
        estmax.add(decision_exp)
        estmax.add(debug)
        maxvalue = estmax.maximize(decision)
        if estmax.check() == sat:
            # print("\nmax sat", maxvalue.value())
            try:
              if float(maxvalue.value().as_fraction()) > threshold:
                  return False # can change class
              else:
                  self.xai_predicted_margin = float(maxvalue.value().as_fraction())
            except:
              print("error max =", maxvalue.value())
              return False
        else:
            print("error")

      if predicted_class == 1:
        estmin = Optimize()
        estmin.add(decision_exp)
        estmin.add(debug)
        minvalue = estmin.minimize(decision)
        if estmin.check() == sat:
            # print("\nmin sat", minvalue.value())
            try:
              if float(minvalue.value().as_fraction()) < threshold:
                  return False # can change class
              else:
                  self.xai_predicted_margin = float(minvalue.value().as_fraction())
            except:
              print("error min =", minvalue.value())
              return False
        else:
            print("error")

      if predicted_class == 0:
        self.solvermodel = estmax.model()
      if predicted_class == 1:
        self.solvermodel = estmin.model()
      return True
  
def generate_results(explainer, X_test, y_pred, classification, path, reorder="asc"):
    results = []
    if classification == 0:
        increase_prob = -0.01
    else:
        increase_prob = 0.01
    for i in X_test[y_pred == classification].index:
        sample = X_test.loc[i].values
        xai = explainer.explain_prob(sample, reorder=reorder)
        xaiprob_initial = explainer.xai_predicted_margin
        len_xai_initial = len(xai)

        xai = explainer.explain_prob(sample, reorder=reorder, target_threshold=xaiprob_initial + increase_prob)
        xaiprob_final = explainer.xai_predicted_margin
        len_xai_final = len(xai)

        if round(xaiprob_initial, 2) == round(xaiprob_final, 2):
            len_xai_initial = len_xai_final
        results.append({
            "index": i,
            "class": classification,
            "xaiprob_initial": round(xaiprob_initial, 2),
            "len_xai_initial": len_xai_initial,
            "xaiprob_final": round(xaiprob_final, 2),
            "len_xai_final": len_xai_final
        })
    df_results = pd.DataFrame(results)
    return df_results
    # df_results.to_csv(f'{path}/results_{classification}_{reorder}.csv', index=False) 

# Prepare datasets and models

In [10]:
def prepare_dataset_model_explainer(dataset_name, dataset_params):
    # Fetch data
    dataset = fetch_data(dataset_name)
    X = dataset.drop('target', axis=1)
    y = dataset['target']

    params = dataset_params[dataset_name]

    # Train model
    model = XGBClassifier(**params)
    model.fit(X, y)
    y_pred = model.predict(X)

    # Metrics
    print("Dataset size:", len(X))
    print("Columns:", len(X.columns))
    # print("Accuracy:", accuracy_score(y, y_pred))
    # print("Precision:", precision_score(y, y_pred))
    # print("Recall:", recall_score(y, y_pred))
    print("F1-score:", f1_score(y, y_pred))

    # Explainer
    explainer = XGBoostExplainer(model, X)

    return model, X, y, explainer

def prepare_all_datasets(dataset_names, dataset_params):
    context = {}
    for name in dataset_names:
        print(f"\n--- Preparing {name} ---")
        model, X, y, explainer = prepare_dataset_model_explainer(name, dataset_params)
        context[name] = (model, X, y, explainer)
    return context

In [22]:
dataset_names = ["saheart", "adult", "mushroom", "sonar"]

dataset_params = {
    "saheart": {"n_estimators": 50, "max_depth": 3},
    "adult": {"n_estimators": 30, "max_depth": 3},
    "mushroom": {"n_estimators": 30, "max_depth": 3},
    "sonar": {"n_estimators": 30, "max_depth": 3},
}

dataset_context = prepare_all_datasets(dataset_names, dataset_params)


--- Preparing saheart ---
Dataset size: 462
Columns: 9
F1-score: 0.9144736842105263

--- Preparing adult ---
Dataset size: 48842
Columns: 14
F1-score: 0.9156271028521145

--- Preparing mushroom ---
Dataset size: 8124
Columns: 22
F1-score: 1.0

--- Preparing sonar ---
Dataset size: 208
Columns: 60
F1-score: 1.0


# Check explanations

In [23]:
def get_explain_results(dataset_name):
    model, X, y, explainer = dataset_context[dataset_name]
    y_pred = model.predict(X)

    # Function to explain a sample of a given class
    def explain_for_class(classification):
        # Get a sample of the specified class in predictions
        indices = X[y_pred == classification].index
        if len(indices) == 0:
            print(f"No samples predicted as class {classification}.")
            return
        i = indices[0]
        sample = X.loc[i].values

        # Initial explanation
        margin = model.predict([sample], output_margin=True)[0]
        print(f"\nClass {classification} - Margin: {margin}, Sample index: {i}")

        xai = explainer.explain_prob(sample)
        print("Initial abductive explanation:", xai)
        
        xaiprob_initial = explainer.xai_predicted_margin
        print("Initial predicted margin:", xaiprob_initial)

        # Adjust margin direction
        # increase_prob = -0.01 if classification == 0 else 0.01

        # Explanation with adjusted threshold
        xai_confidence = explainer.explain_prob(sample, target_threshold= margin / 2)
        xaiprob_final = explainer.xai_predicted_margin
        print("Confidence-aware abductive explanation:", xai_confidence)
        print("Final predicted margin:", xaiprob_final)

    # Explain for both classes
    explain_for_class(0)
    explain_for_class(1)


In [24]:
get_explain_results("mushroom")


Class 0 - Margin: -6.846377849578857, Sample index: 0
Initial abductive explanation: [odor == 6, gill-size == 0, spore-print-color == 1]
Initial predicted margin: -3.9321851558
Confidence-aware abductive explanation: [odor == 6, gill-size == 0, spore-print-color == 1]
Final predicted margin: -3.9321851558

Class 1 - Margin: 7.956404685974121, Sample index: 3
Initial abductive explanation: [odor == 4, veil-color == 2, spore-print-color == 3]
Initial predicted margin: 1.6946676614
Confidence-aware abductive explanation: [odor == 4, gill-spacing == 0, gill-size == 0, veil-color == 2, spore-print-color == 3]
Final predicted margin: 5.4277561085


In [25]:
get_explain_results("adult")


Class 0 - Margin: -0.020596586167812347, Sample index: 4
Initial abductive explanation: [fnlwgt == 338409, sex == 0, workclass == 4, capital-loss == 0, occupation == 10, age == 28, hours-per-week == 40, capital-gain == 0, marital-status == 2, education-num == 13, relationship == 5]
Initial predicted margin: -0.02059663832
Confidence-aware abductive explanation: [fnlwgt == 338409, sex == 0, workclass == 4, capital-loss == 0, occupation == 10, age == 28, hours-per-week == 40, capital-gain == 0, marital-status == 2, education-num == 13, relationship == 5]
Final predicted margin: -0.02059663832

Class 1 - Margin: 2.4348256587982178, Sample index: 0
Initial abductive explanation: [capital-loss == 0, capital-gain == 2174, education-num == 13, relationship == 1]
Initial predicted margin: 0.14839767258
Confidence-aware abductive explanation: [capital-loss == 0, hours-per-week == 40, capital-gain == 2174, marital-status == 4, education-num == 13, relationship == 1]
Final predicted margin: 1.31

In [26]:
get_explain_results("sonar")


Class 0 - Margin: -5.826420783996582, Sample index: 0
Initial abductive explanation: [A51 == 0.013, A31 == 0.3934, A37 == 0.0702, A52 == 0.012, A48 == 0.1037, A45 == 0.1814, A20 == 0.6936, A12 == 0.3553, A34 == 0.114, A16 == 0.2794, A11 == 0.3188]
Initial predicted margin: -0.05964847832
Confidence-aware abductive explanation: [A36 == 0.008, A23 == 0.8203, A9 == 0.1865, A17 == 0.287, A51 == 0.013, A42 == 0.102, A31 == 0.3934, A37 == 0.0702, A52 == 0.012, A15 == 0.178, A48 == 0.1037, A45 == 0.1814, A20 == 0.6936, A12 == 0.3553, A47 == 0.1688, A34 == 0.114, A16 == 0.2794, A11 == 0.3188]
Final predicted margin: -3.00395026691

Class 1 - Margin: 5.359158992767334, Sample index: 12
Initial abductive explanation: [A23 == 0.2605, A44 == 0.0942, A48 == 0.0469, A45 == 0.084, A12 == 0.1102, A21 == 0.2005, A47 == 0.0342, A5 == 0.0357, A4 == 0.0139, A11 == 0.1203]
Initial predicted margin: 0.08716987994
Confidence-aware abductive explanation: [A28 == 0.6742, A10 == 0.0973, A54 == 0.0052, A27 == 0

In [27]:
get_explain_results("saheart")


Class 0 - Margin: -0.2692931890487671, Sample index: 1
Initial abductive explanation: [Sbp == 144, Obesity == 28.87, Adiposity == 28.61, Ldl == 4.41, Typea == 55, Tobacco == 0.01, Famhist == 0]
Initial predicted margin: -0.1002989025
Confidence-aware abductive explanation: [Sbp == 144, Obesity == 28.87, Alcohol == 2.06, Adiposity == 28.61, Ldl == 4.41, Typea == 55, Tobacco == 0.01, Famhist == 0]
Final predicted margin: -0.26929319084

Class 1 - Margin: 1.7746528387069702, Sample index: 0
Initial abductive explanation: [Alcohol == 97.2, Ldl == 5.73, Tobacco == 12, Famhist == 1, Age == 52]
Initial predicted margin: 0.12975603705
Confidence-aware abductive explanation: [Obesity == 25.3, Alcohol == 97.2, Ldl == 5.73, Tobacco == 12, Famhist == 1, Age == 52]
Final predicted margin: 1.07581014872


# Check threshold datasets

## functions

In [28]:
def get_explain_results_n(dataset_name, n_samples):
    model, X, y, explainer = dataset_context[dataset_name]
    y_pred = model.predict(X)

    # Prepare result DataFrame
    results = []

    # Loop over both classes
    for classification in [0, 1]:
        # Get indices for predicted samples of this class
        indices = X[y_pred == classification].index[:n_samples]
        if len(indices) == 0:
            print(f"No samples predicted as class {classification}.")
            continue

        for idx in indices:
            sample = X.loc[idx].values

            # Initial explanation
            margin = model.predict([sample], output_margin=True)[0]
            xai_initial = explainer.explain_prob(sample)
            xaiprob_initial = explainer.xai_predicted_margin
            exp_len_initial = len(xai_initial)

            # Explanation with adjusted threshold (half the margin)
            xai_confidence_case1 = explainer.explain_prob(sample, target_threshold=margin / 2)
            xaiprob_case1 = explainer.xai_predicted_margin
            exp_len_case1 = len(xai_confidence_case1)
            
            increase_prob = -0.01 if margin < 0 else 0.01
            xai_confidence_case2 = explainer.explain_prob(sample, target_threshold= xaiprob_initial + increase_prob)
            xaiprob_case2 = explainer.xai_predicted_margin
            exp_len_case2 = len(xai_confidence_case2)

            # Append to results
            results.append({
                'sample_id': idx,
                'class': classification,
                'pred_margin': round(margin, 4),
                'exp_len_inicial': exp_len_initial,
                'xaiprob_inicial': round(xaiprob_initial, 4),
                'exp_len_case1': exp_len_case1,
                'xaiprob_case1': round(xaiprob_case1, 4),
                'exp_len_case2': exp_len_case2,
                'xaiprob_case2': round(xaiprob_case2, 4),
            })

    # Convert to DataFrame
    df_results = pd.DataFrame(results)
    
    return df_results

def summarize_explanations(df_results):
    # Agrupar por classe e calcular as métricas
    summary = df_results.groupby('class').agg(
        pred_margin_mean=('pred_margin', 'mean'),
        pred_margin_std=('pred_margin', 'std'),
        exp_len_inicial_mean=('exp_len_inicial', 'mean'),
        exp_len_inicial_std=('exp_len_inicial', 'std'),
        xaiprob_inicial_mean=('xaiprob_inicial', 'mean'),
        xaiprob_inicial_std=('xaiprob_inicial', 'std'),
        exp_len_case1_mean=('exp_len_case1', 'mean'),
        exp_len_case1_std=('exp_len_case1', 'std'),
        xaiprob_case1_mean=('xaiprob_case1', 'mean'),
        xaiprob_case1_std=('xaiprob_case1', 'std'),
        exp_len_case2_mean=('exp_len_case2', 'mean'),
        exp_len_case2_std=('exp_len_case2', 'std'),
        xaiprob_case2_mean=('xaiprob_case2', 'mean'),
        xaiprob_case2_std=('xaiprob_case2', 'std'),
    ).reset_index()

    # Função para combinar mean ± std com 2 casas decimais
    def format_mean_std(mean, std):
        return f"{mean:.2f} ± {std:.2f}"

    # Aplicar a formatação em cada par de colunas
    formatted = pd.DataFrame()
    formatted['class'] = summary['class']
    formatted['Pred Margin'] = summary.apply(lambda row: format_mean_std(row['pred_margin_mean'], row['pred_margin_std']), axis=1)
    formatted['Exp Len Inicial'] = summary.apply(lambda row: format_mean_std(row['exp_len_inicial_mean'], row['exp_len_inicial_std']), axis=1)
    formatted['ECM Inicial'] = summary.apply(lambda row: format_mean_std(row['xaiprob_inicial_mean'], row['xaiprob_inicial_std']), axis=1)
    formatted['Exp Len case1'] = summary.apply(lambda row: format_mean_std(row['exp_len_case1_mean'], row['exp_len_case1_std']), axis=1)
    formatted['ECM case1'] = summary.apply(lambda row: format_mean_std(row['xaiprob_case1_mean'], row['xaiprob_case1_std']), axis=1)
    formatted['Exp Len case2'] = summary.apply(lambda row: format_mean_std(row['exp_len_case2_mean'], row['exp_len_case2_std']), axis=1)
    formatted['ECM case2'] = summary.apply(lambda row: format_mean_std(row['xaiprob_case2_mean'], row['xaiprob_case2_std']), axis=1)

    return formatted


# datasets

In [29]:
df_saheart = get_explain_results_n("saheart", 100)
df_summary_saheart = summarize_explanations(df_saheart)
df_summary_saheart

,class,Pred Margin,Exp Len Inicial,ECM Inicial,Exp Len case1,ECM case1,Exp Len case2,ECM case2
0,0,-2.04 ± 1.33,6.49 ± 0.93,-0.36 ± 0.31,7.21 ± 0.70,-1.31 ± 0.81,7.21 ± 0.96,-0.70 ± 0.48
1,1,1.35 ± 0.95,6.30 ± 1.31,0.27 ± 0.24,7.10 ± 1.08,0.86 ± 0.56,6.93 ± 1.29,0.55 ± 0.35


In [30]:
df_adult = get_explain_results_n("adult", 100)
df_summary_adult = summarize_explanations(df_adult)
df_summary_adult

,class,Pred Margin,Exp Len Inicial,ECM Inicial,Exp Len case1,ECM case1,Exp Len case2,ECM case2
0,0,-1.64 ± 1.60,7.39 ± 2.47,-0.18 ± 0.19,8.75 ± 2.29,-0.93 ± 0.86,7.99 ± 2.82,-0.39 ± 0.31
1,1,2.52 ± 1.78,5.14 ± 1.41,0.31 ± 0.30,5.78 ± 1.14,1.48 ± 0.99,5.52 ± 1.85,0.63 ± 0.47


In [31]:
df_mushroom = get_explain_results_n("mushroom", 100)
df_summary_mushroom = summarize_explanations(df_mushroom)
df_summary_mushroom

,class,Pred Margin,Exp Len Inicial,ECM Inicial,Exp Len case1,ECM case1,Exp Len case2,ECM case2
0,0,-6.43 ± 1.15,2.89 ± 1.16,-2.66 ± 1.78,3.74 ± 1.76,-3.95 ± 1.01,3.88 ± 1.16,-3.13 ± 2.06
1,1,7.20 ± 1.10,3.58 ± 0.82,1.14 ± 0.61,5.77 ± 1.01,4.35 ± 0.96,4.57 ± 0.76,2.39 ± 1.06


In [32]:
df_sonar = get_explain_results_n("sonar", 100)
df_summary_sonar = summarize_explanations(df_sonar)
df_summary_sonar

,class,Pred Margin,Exp Len Inicial,ECM Inicial,Exp Len case1,ECM case1,Exp Len case2,ECM case2
0,0,-3.93 ± 1.11,13.28 ± 2.63,-0.08 ± 0.07,17.54 ± 2.81,-2.03 ± 0.57,13.28 ± 2.68,-0.21 ± 0.15
1,1,3.79 ± 1.18,14.00 ± 2.16,0.07 ± 0.07,18.45 ± 1.69,1.95 ± 0.59,14.13 ± 2.07,0.16 ± 0.11


# Test robustness

In [33]:
def get_robustness_results_n(dataset_name, n_samples, n_extra_samples, target_class=1,
                              mult_margin=0, noise_level=0.5):
    model, X, y, explainer = dataset_context[dataset_name]

    # Armazena os resultados em lista
    results = []

    count = 0
    for i in range(len(X)):
        if count >= n_samples:
            break

        sample = X.iloc[i]
        pred_class = model.predict([sample])[0]
        if pred_class != target_class:
            continue

        margin = model.predict([sample], output_margin=True)[0]
        explanation = explainer.explain_prob(sample, reorder="asc", target_threshold=margin * mult_margin)

        satisfying_df = generate_samples_for_conditions(X, explanation, n_samples=n_extra_samples, random_state=42)
        noisy_df = apply_noise_to_samples(X, satisfying_df, noise_level=noise_level)
        noisy_pred = model.predict(noisy_df)
        noisy_pred_margin = model.predict(noisy_df, output_margin=True)

        class_0 = np.sum(noisy_pred == 0)
        class_1 = np.sum(noisy_pred == 1)

        results.append({
            "amostra": i,
            "classe_prevista": int(pred_class),
            "classe_0_com_ruido": int(class_0),
            "classe_1_com_ruido": int(class_1),
            # "output_margin_medio": float(np.mean(noisy_pred_margin)),
            # "output_margin_max": float(np.max(noisy_pred_margin)),
            # "output_margin_min": float(np.min(noisy_pred_margin))
        })

        count += 1

    # Converte para DataFrame
    results_df = pd.DataFrame(results)

    # Cálculo de média ± desvio padrão para cada coluna
    mean_0 = results_df["classe_0_com_ruido"].mean()
    std_0 = results_df["classe_0_com_ruido"].std()

    mean_1 = results_df["classe_1_com_ruido"].mean()
    std_1 = results_df["classe_1_com_ruido"].std()

    mean_results = pd.DataFrame([{
        "explanation class": int(pred_class),
        "noise samples": n_extra_samples,
        "noise level": noise_level,
        "classified 0": f"{mean_0:.2f} ± {std_0:.2f}",
        "classified 1": f"{mean_1:.2f} ± {std_1:.2f}"
    }])

    return mean_results


def generate_samples_for_conditions(df, conditions, n_samples=10, random_state=None):
    """
    Gera amostras aleatórias que atendem a um conjunto de condições, variando apenas as outras features.
    """
    np.random.seed(random_state)
    fixed_values = {str(cond.arg(0)): float(cond.arg(1).as_fraction()) for cond in conditions}

    df_variation = df.drop(columns=fixed_values.keys())
    samples = {col: np.random.uniform(df[col].min(), df[col].max(), n_samples) for col in df_variation.columns}
    samples = {key: np.round(value, 2) for key, value in samples.items()}

    for feature, value in fixed_values.items():
        samples[feature] = [value] * n_samples

    generated_df = pd.DataFrame(samples)
    return generated_df[df.columns]


def apply_noise_to_samples(X, df, noise_level):
    df_noisy = df.copy()
    for col in df.columns:
        min_val = X[col].min()
        max_val = X[col].max()
        random_vals = np.random.uniform(min_val, max_val, size=len(df))
        # Interpola entre valor original e valor aleatório
        df_noisy[col] = (1 - noise_level) * df[col] + noise_level * random_vals
    return df_noisy

In [34]:
configs = [
    {"target_class": 0, "mult_margin": 0},
    {"target_class": 0, "mult_margin": 0.5},
    {"target_class": 1, "mult_margin": 0},
    {"target_class": 1, "mult_margin": 0.5},
]

In [35]:
all_results = []
for config in configs:
    df = get_robustness_results_n(
        "saheart", 
        n_samples=100, 
        n_extra_samples=100, 
        target_class=config["target_class"], 
        mult_margin=config["mult_margin"], 
        noise_level=0.1
    )

    df["exp. threshold"] = "0%" if config["mult_margin"] == 0 else "50%"
    all_results.append(df)

results_df = pd.concat(all_results, ignore_index=True)
results_df

,explanation class,noise samples,noise level,classified 0,classified 1,exp. threshold
0,0,100,0.1,92.14 ± 10.95,7.86 ± 10.95,0%
1,0,100,0.1,93.65 ± 11.58,6.35 ± 11.58,50%
2,1,100,0.1,49.57 ± 32.38,50.43 ± 32.38,0%
3,1,100,0.1,50.41 ± 34.78,49.59 ± 34.78,50%


In [36]:
all_results = []
for config in configs:
    df = get_robustness_results_n(
        "adult", 
        n_samples=100,
        n_extra_samples=100, 
        target_class=config["target_class"], 
        mult_margin=config["mult_margin"], 
        noise_level=0.1
    )

    df["target_class"] = config["target_class"]
    df["exp. threshold"] = "0%" if config["mult_margin"] == 0 else "50%"
    all_results.append(df)

results_df = pd.concat(all_results, ignore_index=True)
results_df

,explanation class,noise samples,noise level,classified 0,classified 1,target_class,exp. threshold
0,0,100,0.1,71.25 ± 23.43,28.75 ± 23.43,0,0%
1,0,100,0.1,70.09 ± 23.34,29.91 ± 23.34,0,50%
2,1,100,0.1,31.94 ± 9.07,68.06 ± 9.07,1,0%
3,1,100,0.1,31.94 ± 10.12,68.06 ± 10.12,1,50%


In [37]:
all_results = []
for config in configs:
    df = get_robustness_results_n(
        "mushroom", 
        n_samples=100,
        n_extra_samples=100, 
        target_class=config["target_class"], 
        mult_margin=config["mult_margin"], 
        noise_level=0.1
    )

    df["target_class"] = config["target_class"]
    df["exp. threshold"] = "0%" if config["mult_margin"] == 0 else "50%"
    all_results.append(df)

results_df = pd.concat(all_results, ignore_index=True)
results_df

,explanation class,noise samples,noise level,classified 0,classified 1,target_class,exp. threshold
0,0,100,0.1,90.50 ± 24.90,9.50 ± 24.90,0,0%
1,0,100,0.1,90.29 ± 25.53,9.71 ± 25.53,0,50%
2,1,100,0.1,18.32 ± 22.36,81.68 ± 22.36,1,0%
3,1,100,0.1,23.30 ± 26.21,76.70 ± 26.21,1,50%


In [38]:
all_results = []
for config in configs:
    df = get_robustness_results_n(
        "sonar", 
        n_samples=100,
        n_extra_samples=100, 
        target_class=config["target_class"], 
        mult_margin=config["mult_margin"], 
        noise_level=0.1
    )

    df["target_class"] = config["target_class"]
    df["exp. threshold"] = "0%" if config["mult_margin"] == 0 else "50%"
    all_results.append(df)

results_df = pd.concat(all_results, ignore_index=True)
results_df

,explanation class,noise samples,noise level,classified 0,classified 1,target_class,exp. threshold
0,0,100,0.1,100.00 ± 0.00,0.00 ± 0.00,0,0%
1,0,100,0.1,99.99 ± 0.10,0.01 ± 0.10,0,50%
2,1,100,0.1,10.41 ± 14.59,89.59 ± 14.59,1,0%
3,1,100,0.1,5.61 ± 11.18,94.39 ± 11.18,1,50%
